# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [3]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [6]:
#Create cluster
from cassandra.cluster import Cluster
try: 
    cluster = Cluster(['127.0.0.1']) 
    session = cluster.connect()
except Exception as e:
    print(e)
    

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [7]:
# Create Keyspace
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [8]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [41]:
# create artist_song_by_sessionid_item 
# using sessionId to unique key and itemInSession clustering columns
query = "create table if not exists artist_song_by_sessionid_item "
query = query + "(sessionId int,\
                    itemInSession int,\
                    artist text, \
                    song text, \
                    lenght float,\
                    primary key(sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)
                     

In [42]:
# load event_datafile_new.csv
file = 'event_datafile_new.csv'

# open and read file
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_song_by_sessionid_item (artist,song,lenght,itemInSession,sessionId) "
        query = query + "VALUES (%s, %s, %s,%s, %s)"
        session.execute(query, (line[0], line[9], float(line[5]), int(line[3]), int(line[8])))

#### Do a SELECT to verify that the data have been inserted into each table

In [43]:
# select by filter by unique key sessionId=338 and itemInSession = 4
query = "select artist, song , lenght from artist_song_by_sessionid_item WHERE sessionId = 338 and itemInSession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
# print row
for row in rows:
    print (row.artist, row.song, row.lenght)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [44]:
# create user_artist_song_by_userid_sessionid_item
# using sessionId, userid to unique key and itemInSession clustering columns
query = "create table if not exists user_artist_song_by_userid_sessionid_item "
query = query + "(sessionId int,\
                    userid int, \
                    itemInSession int,\
                    artist text,\
                    song text,\
                    lastName text,\
                    firstName text, \
                    primary key((sessionId, userid), itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)
    
# load event_datafile_new.csv  
file = 'event_datafile_new.csv'

# open and read file
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_artist_song_by_userid_sessionid_item(artist, song, itemInSession, sessionId, userid, lastName, firstName) "
        query = query + "VALUES (%s, %s, %s, %s, %s,%s, %s)"
    
        session.execute(query, (line[0], line[9], int(line[3]), int(line[8]), int(line[10]), line[4], line[1]))
        
        
# select  by filter by unique key userid = 10, sessionid = 182 order by clustering columns itemInSession      
query = "select artist, song, lastname, firstname  from user_artist_song_by_userid_sessionid_item WHERE sessionId=182 and  userid = 10 order by itemInSession"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
# Print row
for row in rows:
    print (row.artist, row.song, row.lastname, row.firstname)    
                    

Down To The Bone Keep On Keepin' On Cruz Sylvie
Three Drives Greece 2000 Cruz Sylvie
Sebastien Tellier Kilometer Cruz Sylvie
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Cruz Sylvie


In [45]:
# Create user_by_song
# using song to unique key and userid clustering columns
query = "create table if not exists user_by_song "
query = query + "(song text, \
                    userid int,\
                    lastName text,\
                    firstName text,\
                    primary key(song, userid))"
try:
    session.execute(query)
except Exception as e:
    print(e)
    
    
# load file csv  
file = 'event_datafile_new.csv'

# open and read file
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        query = "INSERT INTO user_by_song(userid,song,lastName,firstName) "
        query = query + "VALUES (%s, %s, %s,%s)"

        session.execute(query, (int(line[10]), line[9], line[4], line[1]))
        
        
# Select by filter by unique key song = "All Hands Against His Own" 
query = "select lastName, firstName from user_by_song WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
# Print row
for row in rows:
    print (row.lastname, row.firstname)  
    
                    

Lynch Jacqueline
Levine Tegan
Johnson Sara


### Drop the tables before closing out the sessions

In [46]:
# Drop table 1
query = "drop table artist_song_by_sessionid_item"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
# Drop table 2    
query = "drop table user_artist_song_by_userid_sessionid_item"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
# Drop table 3    
query = "drop table user_by_song"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [47]:
session.shutdown()
cluster.shutdown()